In [2]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross, MeanReversionLongOnly
from src.utils import load_data
warnings.filterwarnings('ignore')

In [4]:
ticker = "IAG.L"
period = "2y"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [8]:
bt_sma = Backtest(data, SmaCross, cash=10_000, commission=0.004, trade_on_close=False)
stats = bt_sma.run()
stats

Start                     2022-03-28 00:00...
End                       2024-03-27 00:00...
Duration                    730 days 01:00:00
Exposure Time [%]                        20.8
Equity Final [$]                  6596.472065
Equity Peak [$]                       10000.0
Return [%]                         -34.035279
Buy & Hold Return [%]               22.570042
Return (Ann.) [%]                  -18.916384
Volatility (Ann.) [%]               12.735985
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -34.035279
Avg. Drawdown [%]                  -34.035279
Max. Drawdown Duration      153 days 00:00:00
Avg. Drawdown Duration      153 days 00:00:00
# Trades                                    3
Win Rate [%]                              0.0
Best Trade [%]                     -10.084089
Worst Trade [%]                    -17.522725
Avg. Trade [%]                    

In [9]:
bt_sma.plot()

Row(id='1496', ...)

In [10]:
%%time
stats = bt_sma.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

CPU times: user 2.24 s, sys: 12.9 ms, total: 2.25 s
Wall time: 2.26 s


Start                     2022-03-28 00:00...
End                       2024-03-27 00:00...
Duration                    730 days 01:00:00
Exposure Time [%]                        97.2
Equity Final [$]                 14259.462974
Equity Peak [$]                  23725.165494
Return [%]                           42.59463
Buy & Hold Return [%]               22.570042
Return (Ann.) [%]                   19.582602
Volatility (Ann.) [%]               41.735972
Sharpe Ratio                         0.469202
Sortino Ratio                        0.868426
Calmar Ratio                          0.42302
Max. Drawdown [%]                  -46.292419
Avg. Drawdown [%]                   -6.300623
Max. Drawdown Duration      363 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                   51
Win Rate [%]                        37.254902
Best Trade [%]                      32.301509
Worst Trade [%]                     -9.494097
Avg. Trade [%]                    

In [11]:
bt_sma.plot()

Row(id='2282', ...)

In [ ]:
%%time
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0, trade_on_close=False)
stats = bt.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

In [ ]:
bt_momentum.plot()

In [ ]:
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0, trade_on_close=True)
stats = bt.run()
stats